In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda

load_dotenv()

chat = ChatOpenAI(model="gpt-3.5-turbo")

# 첫 번째 체인: 언어를 받아 시를 만듦
template_make = ChatPromptTemplate.from_messages([
    ("system", "당신은 프로그래밍 언어를 가지고 훌륭한 시를 만드는 시인입니다. 서정적이면서도 아름다운 시를 만듭니다"),
    ("user", "{language} 언어를 사용해서 프로그래밍 관련 시를 만들어주세요.")
])
chain1 = template_make | chat


# 두 번째 체인: 시를 받아 해석함
template_trans = ChatPromptTemplate.from_messages([
    ("system", "당신은 훌륭한 해석가입니다. 주어진 시를 가지고 논리적이고도 아름답게 해석을 합니다"),
    ("user", "{poem} 시를 해석해주세요.")
])
chain2 = template_trans | chat

# 최종 체인
final_chain = {"poem": chain1} | chain2

# 실행
result = final_chain.invoke({"language": "Python"})
print(result)

content='주어진 시는 파이썬 프로그래밍 언어에 대한 아름다움을 표현한 것으로 보입니다. 시인은 코드의 각 줄이 마치 춤을 추는 것처럼 느껴진다고 표현하며, 들여쓰기가 아름다운 모습을 만들어낸다고 묘사하고 있습니다. 파이썬의 문법은 명료하면서도 우아하며, 문제 해결을 위한 지혜가 담겨 있다고 설명하고 있습니다. 코드가 감각적으로 흐르며 파이썬의 미학을 느낄 수 있다고 표현하고 있습니다.\n\n또한, 테스트와 디버깅을 거친 완성된 프로그램이 탄생할 때 파이썬의 마력을 느낄 수 있으며, 이를 통해 개발의 즐거움을 깨달을 수 있다고 표현하고 있습니다. 마지막으로, 파이썬의 세계에 빠져들면 코드를 작성할 때 느끼는 창조적인 희열과 만족감을 언급하며, 파이썬과 함께하는 시간이 얼마나 소중하고 의미 있는 경험인지 강조하고 있습니다. 이 시는 파이썬의 매력과 개발의 즐거움에 대한 희망과 사랑을 담고 있는 아름다운 시입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 437, 'prompt_tokens': 602, 'total_tokens': 1039, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-C0quwE4ACOdbyezFMuA27cXHdfN1v', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run